## Cleanup Process

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from copy import deepcopy

### Dataframe Initialization and Preliminary Analysis

In [23]:
# Initializing the pandas dataframe
df = pd.read_csv('metadata_3d.csv')

# Number of columns for each data type
print("\nNumber of Columns for Each Data Type:")
print(df.dtypes.value_counts())

print('-----------------------------------')

# Only showing columns with missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # Filter to show only columns with missing values
print("\nColumns with Missing Values:")
print(missing_values)

print('-----------------------------------')

# Number of unique values per column
print("\nNumber of Unique Values per Column:")
print(df.nunique())

print('-----------------------------------')

df.head(8)


Number of Columns for Each Data Type:
float64    108
object       2
int64        1
Name: count, dtype: int64
-----------------------------------

Columns with Missing Values:
Series([], dtype: int64)
-----------------------------------

Number of Unique Values per Column:
patient_id                    46
nodule_idx                    15
malignancy                     9
is_cancer                      3
original_shape_Elongation    164
                            ... 
original_ngtdm_Busyness      164
original_ngtdm_Coarseness    164
original_ngtdm_Complexity    164
original_ngtdm_Contrast      164
original_ngtdm_Strength      164
Length: 111, dtype: int64
-----------------------------------


,patient_id,nodule_idx,malignancy,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LIDC-IDRI-0001,0,5.0,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,LIDC-IDRI-0002,0,4.5,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,LIDC-IDRI-0003,0,2.0,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,LIDC-IDRI-0003,1,4.5,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,LIDC-IDRI-0003,2,3.5,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138
5,LIDC-IDRI-0003,3,3.5,Ambiguous,0.875257,0.787447,10.559810,13.410181,15.416625,14.326133,...,386.183614,0.008275,5.785231,0.497159,71.737012,0.163753,0.005037,3072.604896,0.778994,4.589325
6,LIDC-IDRI-0004,0,1.0,False,0.689709,0.566921,3.500944,6.175367,6.473255,6.262191,...,847.882172,0.020981,5.446971,0.835616,0.387530,0.025053,0.040887,10201.788955,2.680423,53.831661
7,LIDC-IDRI-0005,0,3.0,Ambiguous,0.864986,0.779798,4.126333,5.291539,6.393373,5.186490,...,53.365395,0.048144,4.701746,0.535211,1.272161,0.084420,0.075741,438.266931,0.443285,10.228348


A preliminary examination of our dataframe indicates that it contains a significant number of columns, primarily consisting of floating-point values. Currently, there are no missing values; however, we need to conduct a more in-depth analysis to explore the distribution of data, identify potential outliers, examine correlations between variables, and assess overall data quality.

This Jupyter notebook allows you to review all the modifications made to the dataset.

We will now apply various data analysis techniques to assess the relevance and utility of these columns.

### Simple Considerations

First, we will identify and remove obviously irrelevant columns:

 - `nodule_idx` does not hold significant relevance for our analysis.
 - Since `is_cancer` captures the essential aspect of `malignancy` and will serve as our target variable, we can exclude the `malignancy` column.

Additionally, we determined that adding a new column to represent the total number of nodules per patient could be informative. This new feature is important as it might indicate whether having multiple nodules increases the likelihood of some being cancerous, or perhaps raises the overall risk.

In [24]:
df.drop("nodule_idx", axis=1, inplace=True)
df.drop("malignancy", axis=1, inplace=True)


# Creates a new column that counts the occurrences of each patient_id
df['patient_nodule_count'] = df.groupby('patient_id').cumcount() + 1
insert_position = df.columns.get_loc('patient_id') +1
# Places new column in front of patient_id
df.insert(insert_position, 'patient_nodule_count', df.pop('patient_nodule_count'))

df.head()

,patient_id,patient_nodule_count,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LIDC-IDRI-0001,1,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,26.977005,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,LIDC-IDRI-0002,1,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,37.724947,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,LIDC-IDRI-0003,1,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,25.552264,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,LIDC-IDRI-0003,2,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,24.750701,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,LIDC-IDRI-0003,3,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,12.934109,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


Next, we will eliminate any columns that contain only a single unique value or duplicate the exact values found in another column.

In [25]:
df = df.loc[:, df.nunique() > 1]
df = df.loc[:,~df.T.duplicated()] # treats columns as lines in order to find duplicates
df.head()

,patient_id,patient_nodule_count,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LIDC-IDRI-0001,1,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,26.977005,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,LIDC-IDRI-0002,1,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,37.724947,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,LIDC-IDRI-0003,1,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,25.552264,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,LIDC-IDRI-0003,2,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,24.750701,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,LIDC-IDRI-0003,3,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,12.934109,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


### Correlation


To assess the degree of correlation between the columns and the target variable, and gain better insight about the feature's importance, we decided to create a new DataFrame. We chose the interval of 0.55 and -0.55 as it effectively captures some correlation, both positive and negative. That interval can be changed below.

However, to facilitate this analysis, we must first convert the `patient_id` and `is_cancer` columns into tangible numerical formats. This conversion is essential because correlation calculations require numeric inputs to assess the relationship between variables accurately. 

In [26]:
# Extract the last 4 characters from patient_id and convert to integer
df['patient_id'] = df['patient_id'].str[-4:].astype(int)

# Map is_cancer values to numeric representations
df['is_cancer'] = df['is_cancer'].map({'False': 0, 'Ambiguous': 1, 'True': 2})

df.head()

,patient_id,patient_nodule_count,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,1,1,2,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,26.977005,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,2,1,2,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,37.724947,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,3,1,0,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,25.552264,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,3,2,2,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,24.750701,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,3,3,1,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,12.934109,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


We are now able to compute the correlation matrix effectively and gain insights into the relationships within the dataset:

In [27]:
df2 = deepcopy(df)
correlation_matrix = df2.corr()
is_cancer_corr = correlation_matrix['is_cancer'] 

# Identifying strong correlations with 'is_cancer'
strong_corr_with_is_cancer = is_cancer_corr[(is_cancer_corr > 0.55) | (is_cancer_corr < -0.55)] # can be changed later

# Dropping the 'is_cancer' entry from the results
strong_corr_with_is_cancer = strong_corr_with_is_cancer.drop('is_cancer')
print('There are', strong_corr_with_is_cancer.shape[0], 'columns with a correlation in that range.')
strong_corr_with_is_cancer.head(20)

There are 2 columns with a correlation in that range.


original_shape_SurfaceVolumeRatio   -0.560095
original_glcm_Imc1                   0.608644
Name: is_cancer, dtype: float64

We only want to retain the `patient_id`, `patient_nodule_count`, and `is_cancer` columns, along with those that have demonstrated strong correlation values.

In [28]:
# List of columns to keep
columns_to_keep = ['patient_id', 'patient_nodule_count', 'is_cancer'] + strong_corr_with_is_cancer.index.tolist()

# New DataFrame with the selected columns
filtered_df = df[columns_to_keep]

filtered_df.head()

,patient_id,patient_nodule_count,is_cancer,original_shape_SurfaceVolumeRatio,original_glcm_Imc1
0,1,1,2,0.368609,-0.139251
1,2,1,2,0.388414,-0.206279
2,3,1,0,0.472904,-0.138308
3,3,2,2,0.361115,-0.133980
4,3,3,1,0.816013,-0.386747


In [29]:
#########################################
#possivelmente comentamos algo como, since the correlations are that well defined, to choose another way of selecting cols, we will use:
#########################################

### 2. Recursive Feature Elimination (RFE)

 **Recursive Feature Elimination** (RFE) is a feature selection method that works by **recursively eliminating less important features** to help improve model performance and reduce dimensionality. 
    
It involves training a machine learning model, in this case, a **Random Forest Classifier** and using its performance to evaluate the feature's importance. It starts by training the model on the full set of features.
     
After training, the model assigns importance scores to each feature, based on how much they contribute to reduce the impurity or error when used in splits.
     

In [40]:
X = df.drop(columns=['is_cancer'])  # All columns except the target
y = df['is_cancer']  # Target variable

# Train-test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate RF model
model = RandomForestClassifier(random_state=42)

# Apply RFE to select the top N features
rfe = RFE(estimator=model, n_features_to_select=50)  # Select top N features
X_train_rfe = rfe.fit_transform(X_train, y_train)  # Fit RFE and transform training data
X_test_rfe = rfe.transform(X_test)  # Transform test data using the selected features

# Fit Random Forest model on the selected features
model.fit(X_train_rfe, y_train)

# Predict on the test set
y_pred = model.predict(X_test_rfe)

'''
# Print the selected columns
selected_columns = X.columns[rfe.support_]
print("Selected Features:", selected_columns)
'''

Accuracy: 0.7576


'\n# Print the selected columns\nselected_columns = X.columns[rfe.support_]\nprint("Selected Features:", selected_columns)\n'

Then we can **evaluate the performance of the dataset after the RFE**, when the random forest is applied.

In [42]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score


# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report (includes Precision, Recall, and F1 Score)
class_report = classification_report(y_test, y_pred, target_names=['False', 'Ambiguous', 'True'])
print("\nClassification Report:")
print(class_report)

# Macro-Averaged F1 Score
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"Macro-Averaged F1 Score: {f1_macro:.4f}")

# Weighted-Averaged F1 Score
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted-Averaged F1 Score: {f1_weighted:.4f}")


# ROC-AUC score (One-vs-Rest for multi-class classification)
# We need the probability estimates for calculating ROC-AUC.
y_pred_proba = model.predict_proba(X_test_rfe)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
print(f"ROC-AUC Score (One-vs-Rest): {roc_auc:.4f}")


Confusion Matrix:
[[10  5  0]
 [ 1 11  2]
 [ 0  0  4]]

Classification Report:
              precision    recall  f1-score   support

       False       0.91      0.67      0.77        15
   Ambiguous       0.69      0.79      0.73        14
        True       0.67      1.00      0.80         4

    accuracy                           0.76        33
   macro avg       0.75      0.82      0.77        33
weighted avg       0.79      0.76      0.76        33

Macro-Averaged F1 Score: 0.7675
Weighted-Averaged F1 Score: 0.7577
ROC-AUC Score (One-vs-Rest): 0.8993


For reference, the confusion matrix is formatted as shown below.

| Actual \ Predicted | False (0) | Ambiguous (1) | True (2) |
|--------------------|-----------|---------------|----------|
| **False (0)**       | .       | .             | .       |
| **Ambiguous (1)**   | .         | .            | .        |
| **True (2)**        |      .| .            | .        |




Notice that, to reduce the number of features, we could also use **Principal Component Analysis** (PCA). However, although both PCA and RFE aim to reduce the feature set, they work in fundamentally different ways:

- **RFE** (Recursive Feature Elimination) reduces the number of features by **recursively selecting the most important ones** based on their contribution to the model's performance.
- **PCA** (Principal Component Analysis) reduces dimensionality by **creating new features**, which are **linear combinations of the original ones**, and selects the components that capture the most variance.

In this case, since it makes sense to select features based on their relevance to the end result (target variable), we have opted for the first approach, **RFE**. Nevertheless, **part of the** implementation of PCA is also provided below for reference.

```py

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# StandardScaler() is important for PCA. 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.95)  # Retains 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Print the number of components selected by PCA
print(f"Number of original components: {X_train.shape[1]}")
print(f"Number of PCA components selected: {X_train_pca.shape[1]}")

```

## Modeling

In [ ]:
############# a partir daqui e preciso ver se se faz aqui, se se importa para o report somehow, etc... ############

In [35]:
'''
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
'''

'\nfrom sklearn.preprocessing import LabelEncoder\n\nlabel_encoder = LabelEncoder()\ny_train = label_encoder.fit_transform(y_train)\ny_test = label_encoder.transform(y_test)\n'

In [36]:
'''
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Initialize the XGBoost model with regularization
xgb_model = XGBClassifier(random_state=42, reg_alpha=0.01, reg_lambda=0.01)

# Step 8: Define hyperparameter search space for tuning
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Step 9: Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_train_pca, y_train) ####################################################################################### fiz isto para tentar ver se e pertinente dar tuning na NN, isto serviu de exemplo

# Use the best estimator found by GridSearchCV, not working as I want but might not be necessary
best_xgb_model = grid_search.best_estimator_

# check performance
y_pred = best_xgb_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

# Train and test using original (scaled) data without PCA to compare
grid_search.fit(X_train_scaled, y_train)

print(f"XGBoost model accuracy with PCA: {accuracy}")

import matplotlib.pyplot as plt
from xgboost import plot_importance

# Assuming grid_search.best_estimator_ is your trained XGBoost model
xgb_model = grid_search.best_estimator_

# Plot feature importance
plt.figure(figsize=(10, 8))
plot_importance(xgb_model, max_num_features=10)  # Adjust max_num_features to show more or fewer
plt.title('Feature Importance')

# Save the plot as a file (PNG in this example)
plt.savefig('xgboost_feature_importance.png', dpi=300, bbox_inches='tight')

plt.close()


SyntaxError: incomplete input (1043158175.py, line 1)

In [ ]:
import pandas as pd
import numpy as np

# Assuming `pca` is your PCA object and `X_train` is your original feature set
pca_components = pca.components_

# Create a DataFrame for better visualization
feature_names = X_train.columns
pca_df = pd.DataFrame(data=pca_components, columns=feature_names)

# Display the contributions of each feature to the PCA components
print(pca_df)
